Süper—hem kodu **adım adım** açıklayayım, hem de **hata veren yerleri** modern PyTorch ile **düzelten** küçük bir örnek kod bırakayım. Önce senin kodunun mantığı:

---

## Neden PyTorch? Neden `class` (nn.Module)?

* **PyTorch** otomatik türev (**autograd**) ve GPU hızlandırma verir; tensor operasyonlarıyla ileri/geri yayılımı kendisi çıkarır.
* Bir ağı **`nn.Module` sınıfı** olarak yazınca:

  * Parametreler otomatik **kayıt** edilir (optimizer kolay).
  * **`forward`** ile akışı tanımlarsın; geri yayılımı PyTorch hesaplar.
  * **`.to(device)`** ile komple modeli GPU/CPU’ya taşımak basit olur.

---

## Kodun aşama aşama açıklaması

### 1) Veri hazırlığı

```python
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t').values
test_set     = pd.read_csv('ml-100k/u1.test',  delimiter = '\t').values
nb_users  = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
```

* **u1.base/u1.test** satırları: `(user_id, movie_id, rating, timestamp)`.
* Kullanıcı ve film sayısını buluyoruz.

```python
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies  = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
```

* Her kullanıcı için **yoğun (dense)** bir **kullanıcı×film** vektörü oluşturuyor (oylanmamış filmler `0` kalıyor).

```python
training_set = torch.FloatTensor(convert(training_set))
test_set     = torch.FloatTensor(convert(test_set))
```

### 2) Stacked Autoencoder (SAE) mimarisi

```python
class SAE(nn.Module):
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)     # bottleneck (latent 10)
        self.fc3 = nn.Linear(10, 20)     # decoder
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)                  # çıkışta aktivasyon yok (lineer)
        return x
```

* **Encoder:** nb\_movies → 20 → 10
* **Decoder:** 10 → 20 → nb\_movies
* Bu bir **undercomplete AE** (sıkıştırma: 10 boyut).
* Aktivasyonlar `Sigmoid`. (Çıkış katmanında aktivasyon yok.)

> Not: Çıktı **0–5** aralığındaki ratingleri yeniden kuracaksa, iki seçenekten biri daha doğal olur:
>
> * Ratingleri **\[0,1]** ölçeğine çekip **çıktıya `Sigmoid`** koymak, **veya**
> * Ratingleri **\[0,5]** bırakıp **çıktıyı lineer** tutmak (şu anki gibi).
>   Eğitimin stabil olması için genelde **ölçekleme önerilir** (aşağıda vereceğim örnek kodda ölçekli versiyonu göstereceğim).

### 3) Eğitim döngüsü (kullanıcı bazlı, maskeleme)

* Girdi `input = training_set[id_user].unsqueeze(0)` (tek kullanıcı = tek batch).

* `target = input.clone()` (rekonstrüksiyon hedefi input’un kendisi).

* **Maskeleme:** Kullanıcı hangi filmlere oy verdiyse (target > 0) sadece **onlarda** hatayı hesaba katmak istiyoruz. Bu yüzden:

  ```python
  output = sae(input)
  output[target == 0] = 0  # oylanmamış (0) yerlerde hatayı sıfırla
  loss = criterion(output, target)
  ```

* Ortalama hata adil olsun diye, **oylanan film sayısına** göre düzeltiyor:

  ```python
  mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
  ```

  Sonra **RMSE** benzeri bir rapor için `np.sqrt(loss * mean_corrector)` ekleniyor.

* **Optimizer:** `RMSprop(lr=0.01, weight_decay=0.5)` (weight\_decay=0.5 modern standartlara göre epey büyük; tipik değerler 1e-5–1e-3 arasıdır.)

---

## Hatanın sebebi ve çözümü

Hata:

```
IndexError: invalid index of a 0-dim tensor.
train_loss += np.sqrt(loss.data[0]*mean_corrector)
```

* Yeni PyTorch sürümlerinde `loss.data` **0-boyutlu tensordur**; `[0]` ile indekslenmez.
* **Çözüm:** `.item()` kullan:

  ```python
  train_loss += np.sqrt(loss.item() * mean_corrector)
  ```

Ayrıca **modern PyTorch kalıbı**:

* `Variable` artık kullanılmıyor (Tensor yeterli).
* Her iterasyonda **`optimizer.zero_grad()` → `loss.backward()` → `optimizer.step()`** sırası.
* `target.require_grad = False` (typo) yerine `target.requires_grad_(False)` veya hiç gerek yok.

---

## Daha temiz ve modern örnek (maskeli MSE + ölçekleme)

Aşağıdaki örnek:

* Ratingleri **\[0,1]** aralığına ölçekler (`/5.0`),
* **Maskeli MSE** ile sadece **oylanan** yerlerde loss hesaplar,
* `optimizer.zero_grad()` ve `.item()` kullanır,
* Testte **maskeli RMSE** raporlar.

```python
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# ---------- Veri yükleme ve dönüştürme ----------
def load_ml100k_matrix(base_path="ml-100k"):
    tr = pd.read_csv(f"{base_path}/u1.base", delimiter="\t", header=None, names=["u","i","r","t"]).values
    te = pd.read_csv(f"{base_path}/u1.test", delimiter="\t", header=None, names=["u","i","r","t"]).values
    nb_users  = int(max(tr[:,0].max(), te[:,0].max()))
    nb_movies = int(max(tr[:,1].max(), te[:,1].max()))
    def to_matrix(data):
        X = np.zeros((nb_users, nb_movies), dtype=np.float32)
        for u in range(1, nb_users+1):
            movies  = data[:,1][data[:,0] == u]
            ratings = data[:,2][data[:,0] == u]
            X[u-1, movies-1] = ratings
        return X
    return to_matrix(tr), to_matrix(te)

Xtr, Xte = load_ml100k_matrix()

# Ölçekleme [0,1]; mask (oylananlar)
def binarize_or_scale(X):
    mask = (X > 0).astype(np.float32)     # bilinen yerler
    Xsc  = (X / 5.0).astype(np.float32)   # 0..5 -> 0..1
    return Xsc, mask

Vtr, Mtr = binarize_or_scale(Xtr)
Vte, Mte = binarize_or_scale(Xte)

Vtr = torch.from_numpy(Vtr)   # (U, I)
Mtr = torch.from_numpy(Mtr)
Vte = torch.from_numpy(Vte)
Mte = torch.from_numpy(Mte)

device = "cuda" if torch.cuda.is_available() else "cpu"
Vtr, Mtr, Vte, Mte = Vtr.to(device), Mtr.to(device), Vte.to(device), Mte.to(device)

# ---------- Model ----------
class SAE(nn.Module):
    def __init__(self, n_items):
        super().__init__()
        self.fc1 = nn.Linear(n_items, 128)
        self.fc2 = nn.Linear(128, 64)     # bottleneck
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, n_items)
        self.act = nn.ReLU()
        self.out = nn.Sigmoid()           # 0..1 ölçeği için

    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.act(self.fc3(x))
        x = self.out(self.fc4(x))         # 0..1 tahmin
        return x

model = SAE(n_items=Vtr.size(1)).to(device)
opt = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# Maskeli MSE: sadece bilinenlerde hata
def masked_mse(pred, target, mask, eps=1e-8):
    se = ((pred - target) ** 2) * mask
    return se.sum() / (mask.sum() + eps)

# ---------- Eğitim ----------
epochs = 50
model.train()
for ep in range(1, epochs+1):
    opt.zero_grad()
    # (Bu örnekte bütün kullanıcıları tek batch'te veriyoruz; istersen mini-batch yapabilirsin)
    out = model(Vtr)
    loss = masked_mse(out, Vtr, Mtr)
    loss.backward()
    opt.step()

    if ep % 5 == 0 or ep == 1:
        # RMSE raporu (bilinenlerde)
        rmse = torch.sqrt(masked_mse(out, Vtr, Mtr)).item()
        print(f"Epoch {ep:02d} | train RMSE: {rmse:.4f}")

# ---------- Test ----------
model.eval()
with torch.no_grad():
    pred_te = model(Vte)
    test_rmse = torch.sqrt(masked_mse(pred_te, Vte, Mte)).item()
print(f"Test RMSE (known entries): {test_rmse:.4f}")
```

### Neler değişti / neden daha iyi?

* **`Variable` yok**, modern PyTorch tensorları yeterli.
* **`.item()`** kullanıyoruz (0-dim tensor → Python float).
* **`optimizer.zero_grad()`** her iterasyonda çağrılıyor.
* **Maske** ile yalnız **oylanan** yerlerde loss:

  * Senin yaklaşımındaki `output[target==0]=0` fikren aynı sonucu hedefliyor ama **loss’u açıkça maskeyle** yazmak daha temiz, hatasız.
* **Çıkış aktivasyonu Sigmoid** çünkü ratingleri **0..1** ölçeğine aldık.
  (Dilersen ratingleri olduğu gibi tutup **output’u lineer** bırakabilirsin; o zaman kayıplar/öğrenme oranını yeniden ayarlamak gerekir.)

---

## Kısa özet (akılda kalsın)

* **AE**: girdi = çıktı (rekonstrüksiyon), **bottleneck** → temsil öğrenme.
* **Maskeleme** şart: oylanmamış öğelerde kayıp hesaplanmaz.
* **Hatan:** `.data[0]` yerine **`.item()`** kullan.
* **Modern kalıp:** `zero_grad()` → `backward()` → `step()`; `Variable` yok.
* **Ölçekleme** ve/veya **uygun çıkış aktivasyonu** (Sigmoid/Tanh/Linear) seçimi eğitimi çok rahatlatır.

İstersen bunu **mini-batch** (DataLoader) haline çevireyim veya **Sparse AE / Denoising AE** varyantını aynı MovieLens üzerinde de gösterebilirim.
